# Rasterio

- [Rasterio](https://rasterio.readthedocs.io/en/latest/) [[github](https://github.com/mapbox/rasterio)] (and older set of docs https://mapbox.github.io/rasterio/ which was removed). Note, for some reason rtd redirects to the "stable" version, but github links to "latest" which should be preferred!
- https://rasterio.groups.io/g/main support forum
- https://sgillies.net/tags/rasterio.html is good for current news!
- The [Affine](https://github.com/sgillies/affine) library is used for transformations in Rasterio.
- https://github.com/mapbox/rasterio-cookbook (out of date!)


- rasterio depends on GDAL
- there is a command line interface `rio`, which is described in [Command Line User Guide](https://rasterio.readthedocs.io/en/latest/cli.html)

See also shapely and fiona, maybe also cartopy, pyproj, pysal, etc. [GeoRaster](https://georaster.readthedocs.io/en/latest/) [[github](https://github.com/atedstone/georaster)] looks interesting.

todo: read up on this good ref https://chris35wills.github.io/courses/pydata_stack/

## Questions

? verify crs and coord <-> pixel conversion functions and aviris


## Rasterio Tutorials

- https://automating-gis-processes.github.io/CSC18/lessons/L6/overview.html, esp. https://automating-gis-processes.github.io/CSC18/lessons/L6/reading-raster.html


## Importing Rasterio

In [ ]:
%matplotlib inline

In [ ]:
import rasterio

In [ ]:
# Uncomment to examine rasterio package.
# dir(rasterio)
# help(rasterio)

## Rasterio Versions

The 1.0 release was announced in the [Rasterio 1.0.0](https://sgillies.net/2018/07/13/rasterio-1-0-0.html) blog post.

[Migrating to Rasterio 1.0](https://rasterio.readthedocs.io/en/latest/topics/migrating-to-v1.html) outlines the changes.

See [CHANGES.txt](https://github.com/mapbox/rasterio/blob/master/CHANGES.txt) for changelog.

- ...
- `ul()` replaces the `xy()` method, note `xy` defaults to center of pixel while `ul` defaults to upper left of pixel.
- `read()` replaces `read_band()`
- `read_masks()` replaces `read_mask()`

Note: `ul(row, col)` was replaced by `xy(row, col)` in 1.0+, 

### Transform Backstory

Rasterio v0.36 used GDAL geotransform arrays in their native form, but v1.0 moved toward `affine.Affine` instances.
During the change a transitional `affine` attribute was added for the new `Affine` approach and `transform` was deprecated, but by v1.0 `transform` changed to new affine and `affine` was removed.

See [affine.Affine() vs. GDAL-style geotransforms](https://rasterio.readthedocs.io/en/latest/topics/migrating-to-v1.html#affine-affine-vs-gdal-style-geotransforms) (from Migrating to Rasterio 1.0) and the links therein for more on the transform vs. affine drama, as well as  [rasterio#86](https://github.com/mapbox/rasterio/issues/86).


 ## Understanding the Rasterio Codebase
 
 Affine?
 
 Rasterio is confusing.
 
 The basic `rasterio` module is in [`__init__.py`](https://github.com/mapbox/rasterio/blob/master/rasterio/__init__.py)

The [rasterio._base module](https://rasterio.readthedocs.io/en/latest/api/rasterio._base.html) (defined in [`_base.pyx`](https://github.com/mapbox/rasterio/blob/master/rasterio/_base.pyx)) defines numpy-free base classes, including `DatasetBase`. Some methods include `get_crs()`, `get_transform()`, as well as `read_crs()` and `read_transform()` but they are just called by the corresponding `_get_*` methods to set the values of `_crs` and `_transform` if they are not already set. Note "`get_crs`, `set_crs`, `set_nodatavals`, `set_descriptions`, `set_units`, and `set_gcps` are deprecated and will be removed in version 1.0. They have been replaced by fully settable dataset properties crs, nodatavals, descriptions, units, and gcps.", so this section needs to be updated! Also see if/how crs and transform are different than their `read_` methods. And what's up with `get_transform` it's still around.

TODO, it seems that transform attribute is set in `_io.pyx`, but it's not clear where transform is set? Note some set with `@property / def property_name:` and some using `property property_name:`.

So `wkt` seems to be a different representation of the CRS, 

__i think some need to be imported individually!__ e.g. `rasterio.plot`
and the underscore versions are some compiled thing

What's up with gcps and the warp module, and why do DatasetReader have a gcps attribute?

https://rasterio.readthedocs.io/en/latest/api/ lists modules and submodules but not all are available
 
- `compat`
- `coords`
- crs `_crs`
- drivers `_drivers` 
- `dtypes`
- `enums`
- `env`
- `errors`
- features `_features`
- fill `_fill`
- io `_io`?
- mask ?
- merge ?
- plot ?
- `profiles`
- sample ?
- `transform`
- `vfs`
- vrt ?
- warp `_warp`
- `windows`
 
`rasterio.io` is kind of available as `rasterio._io` but not certain


## Rasterio Objects

The [rasterio.io module](https://rasterio.readthedocs.io/en/latest/api/rasterio.io.html) defines Rasterio's high level Dataset Objects:

- `DatasetReader`
- `DatasetWriter`
- `BufferedDatasetWriter`---only writes to disk or network when `close()` is called.
- `MemoryFile`

In addition to inheriting from their Cython Base classes `DatasetReader`/`DatasetWriter`/`BufferedDatasetWriter` also inherit from `WindowMethodsMixin` and `TransformMethodsMixin` (provides `index()` and `xy()` methods).

Other Objects:

- `Band`
- `Window`
- `CRS`
- `Affine`
- ...


In [ ]:
# The rasterio.io module defines 5 Dataset classes.
# Here we investigate the relationship between Dataset classes, including Cython parent classes.

# DatasetReader
assert issubclass(rasterio.io.DatasetReader, rasterio._io.DatasetReaderBase)
assert issubclass(rasterio._io.DatasetReaderBase, rasterio._base.DatasetBase)  # DatasetBase is a subclass of builtins.object.

# DatasetWriter
assert issubclass(rasterio.io.DatasetWriter, rasterio._io.DatasetWriterBase)
assert issubclass(rasterio._io.DatasetWriterBase, rasterio._io.DatasetReaderBase)

# BufferedDatasetWriter
assert issubclass(rasterio.io.BufferedDatasetWriter, rasterio._io.BufferedDatasetWriterBase)
assert issubclass(rasterio._io.BufferedDatasetWriterBase, rasterio._io.DatasetWriterBase)

# MemoryFile
assert issubclass(rasterio.io.MemoryFile, rasterio._io.MemoryFileBase)  # MemoryFileBase is a subclass of builtins.object.

# ZipMemoryFile
assert issubclass(rasterio.io.ZipMemoryFile, rasterio.io.MemoryFile)

In [ ]:
# Get base classes.
rasterio.io.DatasetReader.__bases__

`DatasetReaderBase`, `DatasetWriterBase`, `BufferedDatasetWriterBase`, and `MemoryFileBase` are imported from `rasterio._io` into [`rasterio.io`](https://github.com/mapbox/rasterio/blob/master/rasterio/io.py), where `DatasetReader`, `DatasetWriter`, `MemoryFile`, and `BufferedDatasetReader` are defined.

`DatasetReader` and `MemoryFile` are imported into the library's global symbol table.


In [ ]:
# 
assert rasterio.DatasetReader is rasterio.io.DatasetReader
assert rasterio.MemoryFile is rasterio.io.MemoryFile

# Cython parent classes are imported alongside Python classes.
assert rasterio._io.DatasetReaderBase is rasterio.io.DatasetReaderBase
assert rasterio._io.DatasetWriterBase is rasterio.io.DatasetWriterBase

Compare attributes of Rasterio classes. `DatasetWriter` contains all `DatasetReader` attributes and then some. `BufferedDatasetWriter` contains the exact same attributes as `DatasetWriter`. `MemoryFile` contains a few attributes that `DatasetReader/Writer` do not.

In [ ]:
reader_attributes = set(dir(rasterio.io.DatasetReader))
writer_attributes = set(dir(rasterio.io.DatasetWriter))
bufferedwriter_attributes = set(dir(rasterio.io.BufferedDatasetWriter))
memoryfile_attributes = set(dir(rasterio.io.MemoryFile))

print(writer_attributes - reader_attributes)

# print(writer_attributes ^ bufferedwriter_attributes)

# print(memoryfile_attributes - reader_attributes)
# print(reader_attributes - memoryfile_attributes)
# print(reader_attributes & memoryfile_attributes)

print(memoryfile_attributes - writer_attributes)
# print(writer_attributes - memoryfile_attributes)
# print(writer_attributes & memoryfile_attributes)

In [ ]:
rasterio.io.DatasetReader.xy?

### DatasetReader

Let's focus on reading raster data, and ignore the idea of writing it for the time being.

Dataset is read into a [`DatasetReader`](https://rasterio.readthedocs.io/en/latest/api/rasterio.io.html#rasterio.io.DatasetReader) object (older versions of Rasterio may create a `RasterReader` [ref [rasterio#1221](https://github.com/mapbox/rasterio/issues/1221)]).

There is some GDAL object beneath?

[`rasterio._base.DatasetBase`](https://rasterio.readthedocs.io/en/latest/api/rasterio._base.html#rasterio._base.DatasetBase) > [`rasterio._io.DatasetReaderBase`](https://rasterio.readthedocs.io/en/latest/api/rasterio._io.html#rasterio._io.DatasetReaderBase) > [`rasterio.io.DatasetReader`](https://rasterio.readthedocs.io/en/latest/api/rasterio.io.html#rasterio.io.DatasetReader)

#### DatasetReader Properties

note that some/all are generic dataset attributes also shared by DatasetWriter

- `name`---!
- `files`!
- `mode`---
- `closed`---

- `count`---the number of raster bands in the dataset
- `width`---
- `height`---
- `bounds`---derived from `transform`
- `transform`---affine transformation matrix
- `crs`---
- `meta`
- `profile`!---see [rasterio#406](https://github.com/mapbox/rasterio/pull/406)
- `descriptions`---descriptions for each dataset band, probably set manually.
- `units`---units string for each dataset band, see [pint](https://pint.readthedocs.io/) for suggestions, probably set manually.

- `indexes`
- `dtypes`

- `res`---



### MemoryFile

[`MemoryFile`](https://rasterio.readthedocs.io/en/latest/api/rasterio.io.html#rasterio.io.MemoryFile)

see also MemoryFile? for in-memory? see [In-Memory Files](https://rasterio.readthedocs.io/en/stable/topics/memory-files.html) but these seem to be related to network files or GDAL somethingorother!

See also [`ZipMemoryFile`](https://rasterio.readthedocs.io/en/latest/api/rasterio.io.html#rasterio.io.ZipMemoryFile), a subclass of `MemoryFile`.

Use by creating an instance using `MemoryFile` constructor then return a dataset object with `open()`.

MemoryFile has `open()`, `read()`, `seek()`, `tell()`, etc.


## Open a Raster File

- [`rasterio.open()`](https://rasterio.readthedocs.io/en/latest/api/rasterio.html#rasterio.open) (defined in [`__init__.py`](https://github.com/mapbox/rasterio/blob/master/rasterio/__init__.py)) returns a [**`DatasetReader`**](https://rasterio.readthedocs.io/en/latest/api/rasterio.io.html#rasterio.io.DatasetReader) or [`DatasetWriter`](https://rasterio.readthedocs.io/en/latest/api/rasterio.io.html#rasterio.io.DatasetWriter).
- `rasterio.band()` (defined in [`__init__.py`](https://github.com/mapbox/rasterio/blob/master/rasterio/__init__.py)) allows you to wrap a dataset and and one or more of its bands up into a `rasterio.Band`, which is really just a named tuple with 4 fields:

        Band = namedtuple('Band', ['ds', 'bidx', 'dtype', 'shape'])

It's not clear what you can do with a Band object!?

definitions and inheritance:

- `DatasetReader`, `DatasetWriter`, `MemoryFile`, `BufferedDatasetWriter` are part of the [rasterio.io module](https://rasterio.readthedocs.io/en/latest/api/rasterio.io.html) (defined in [`io.py`](https://github.com/mapbox/rasterio/blob/master/rasterio/io.py), but nothing is really defined until [`_io.pyx`](https://github.com/mapbox/rasterio/blob/master/rasterio/_io.pyx))).
- `DataSetReader` inherits from `rasterio._io.DatasetReaderBase`, `rasterio.windows.WindowMethodsMixin`, `rasterio.transform.TransformMethodsMixin`.
- The [rasterio._io module](https://rasterio.readthedocs.io/en/latest/api/rasterio._io.html) (defined in [`_io.pyx`](https://github.com/mapbox/rasterio/blob/master/rasterio/_io.pyx)) defines `DatasetReaderBase`, `DatasetWriterBase`, `MemoryFileBase`, `InMemoryRaster`, and some writer classes, also includes `read()`.
- `DatasetBase` is defined in [`_base.pyx`](https://github.com/mapbox/rasterio/blob/master/rasterio/_base.pyx). See [rasterio._base module](https://rasterio.readthedocs.io/en/latest/api/rasterio._base.html).

Note with BufferedDatasetWriter data is not written until `close()` is called. What is its use with other.

what the hell is the *band cache*?

open in update mode? r+, w ?

[Accessing datasets located in buffers using MemoryFile and ZipMemoryFile (rasterio#977)](https://github.com/mapbox/rasterio/issues/977) is a nice review of ways to open files.

[Advanced Datasets](https://rasterio.readthedocs.io/en/latest/topics/datasets.html) describes how to open files from the web, zip, etc.! Note that some GDAL drivers can open zip files.

`open_rasterio()` TODO source `transform = Affine.from_gdal(*da.attrs['transform'])` I think is wrong! File issue!


In [ ]:
rasterio.band?

## Read Data into Memory

[Reading Datasets](https://rasterio.readthedocs.io/en/latest/topics/reading.html)

Use [`dataset.read()`](https://rasterio.readthedocs.io/en/latest/api/rasterio.io.html#rasterio.io.DatasetReader.read) (defined on `DatasetReaderBase` in [`_io.pyx`](https://github.com/mapbox/rasterio/blob/master/rasterio/_io.pyx)) to read pixels into a numpy array. Without args it will read the entire dataset, or else specify which band or list of bands to read.

*Rasterio band order* for arrays is: `(bands, rows, columns)`. Use `rasterio.plot.reshape_as_image()` to convert axis order to `(rows, columns, bands)`, and `rasterio.plot.reshape_as_raster()` to convert back. These conversion functions are defined in [`plot.py`](https://github.com/mapbox/rasterio/blob/master/rasterio/plot.py), and use `np.ma.transpose()` and `np.transpose()`, respectively. They likely just create a view and don't copy the data, verify using `strides`.

See [Interoperability](https://rasterio.readthedocs.io/en/latest/topics/image_processing.html).


### windows and blocks

The `read()` method may be used to obtain a view on to a rectangular subset of the dataset, referred to as a *window*. The window may be specified usings offsets, or better with a `Window` object. The [rasterio.windows module](https://rasterio.readthedocs.io/en/latest/api/rasterio.windows.html) (defined in [`window.py`](https://github.com/mapbox/rasterio/blob/master/rasterio/windows.py)) defines the [`Window`](https://rasterio.readthedocs.io/en/latest/api/rasterio.windows.html#rasterio.windows.Window) class and related utility functions.

See [Windowed reading and writing](https://rasterio.readthedocs.io/en/latest/topics/windowed-rw.html) for a good overview of windows and blocks.

Dataset attributes of interest when dealing with blocks are:

- [`block_shapes`](https://rasterio.readthedocs.io/en/latest/api/rasterio.io.html#rasterio.io.DatasetReader.block_shapes)
- [`block_size()`](https://rasterio.readthedocs.io/en/latest/api/rasterio.io.html#rasterio.io.DatasetReader.block_size)
- [`block_window()`](https://rasterio.readthedocs.io/en/latest/api/rasterio.io.html#rasterio.io.DatasetReader.block_window)
- [`block_windows()`](https://rasterio.readthedocs.io/en/latest/api/rasterio.io.html#rasterio.io.DatasetReader.block_windows)

Blocks and Windows are also discussed in [Concurrent processing](https://rasterio.readthedocs.io/en/latest/topics/concurrency.html).

The issue [pangeo-data/pangeo#183](https://github.com/pangeo-data/pangeo/issues/183) touches on how windows and blocks in Rasterio relate to chunks in dask and xarray.


## Spatial Indexing

`xy()`


In [ ]:
# This is boilerplate for spatial indexing, how to use index() to convert from meters relative to CRS origin to raster pixel.

m_east = 0
m_south = 0
x, y = (dataset.bounds.left + m_east, dataset.bounds.top - m_south)
row, col = dataset.index(x, y)
print(row, col)

band1 = dataset.read(1)
# e.g. get pixel value at corresponding pixel
print(band1[row, col])

## bounding box

even if the transform has a rotation, what does that mean for bounds?

## masks and nodata

[Nodata Masks](https://rasterio.readthedocs.io/en/latest/topics/masks.html)

- `nodata`---typically returns the first item in `nodatavals` array
- `nodatavals`?
- `read_masks`?
- `write_masks`?
- `dataset_masks()`?


When `masked=True` the `read()` method will return a masked array.

The [rasterio.mask module](https://rasterio.readthedocs.io/en/latest/api/rasterio.mask.html) may be useful for creating masks from vector geometries.

## Plotting with Rasterio

- [Plotting](https://rasterio.readthedocs.io/en/latest/topics/plotting.html)
- [rasterio.plot module](https://rasterio.readthedocs.io/en/latest/api/rasterio.plot.html) (source [`plot.py`](https://github.com/mapbox/rasterio/blob/master/rasterio/plot.py))

`rasterio.plot.show(source, with_bounds=True, contour=False, contour_label_kws=None, ax=None, title=None, transform=None, adjust='linear', **kwargs)`
is a wrapper for pyplot, I believe imshow but check?

? what about with_bounds?

source can be a Band or tuple (dataset, bdx), or array, or dataset (opened in `r` mode!) in which case the first band is displayed unless `colorinterp` metadata is set - how?

TODO it's really unclear how to plot three bands from a multispectral image! seems possible if you create a band object

The rasterio.plot module defines

- `show()`
- `show_hist()`
- ``
- `reshape_as_image()`
- `reshape_as_raster()`

Note that Rasterio object do not define their own plotting methods.

## Rasterio Modules

It seems that many rasterio submodules must be imported individually, i.e. `import rasterio` does not make them available.


### rasterio.merge module

The [rasterio.merge module](https://rasterio.readthedocs.io/en/latest/api/rasterio.merge.html) defines the `rasterio.merge.merge()` function.

Mosaics files using reverse painters algorithm. All must have same number of bands, data type, and crs. If `bounds` are not specified they are determined from the bounds of the input rasters. If `res` is not specified it is taken from first input raster.

https://automating-gis-processes.github.io/CSC/notebooks/L5/raster-mosaic.html

https://automating-gis-processes.github.io/CSC18/lessons/L6/raster-mosaic.html


In [ ]:
# Must be imported to use.
import rasterio.merge
# help(rasterio.merge)

### rasterio.warp module

[rasterio.warp module](https://rasterio.readthedocs.io/en/latest/api/rasterio.warp.html)

[Reprojection](https://rasterio.readthedocs.io/en/latest/topics/reproject.html)

Even the docs are a little vague about what this stuff actually does, but it seems useful!

- `rasterio.warp.reproject()`
- `rasterio.warp.transform()`
- `rasterio.warp.transform_bounds()`
- `rasterio.warp.calculate_default_transform()`


In [ ]:
# Must be imported to use.
import rasterio.warp
# help(rasterio.warp)

### rasterio.vrt module

[rasterio.vrt module](https://rasterio.readthedocs.io/en/latest/api/rasterio.vrt.html)

[Virtual Warping](https://rasterio.readthedocs.io/en/latest/topics/virtual-warping.html)

Seems to be imported automatically, but where?


In [ ]:
rasterio.vrt?

### rasterio.features module

[rasterio.features module](https://rasterio.readthedocs.io/en/latest/api/rasterio.features.html)

The [Vector Features](https://rasterio.readthedocs.io/en/latest/topics/features.html) Topic discusses the use of:

- shapes()
- rasterize()

additional functions in the features module:

- `bounds()`
- `dataset_features()`
- `geometry_mask()`
- `geometry_window()`
- `is_valid_geom()`
- `sieve()`


In [ ]:
import pprint
import rasterio
from rasterio import features

with rasterio.open('../13547682814_f2e459f7a5_o.png') as src:
    blue = src.read(3)

mask = blue != 255
shapes = features.shapes(blue, mask=mask)
# pprint.pprint(next(shapes))

In [ ]:
with rasterio.open('../13547682814_f2e459f7a5_o.png') as src:
    image = features.rasterize(
            ((g, 255) for g, v in shapes),
            out_shape=src.shape)

### rasterio.crs module

The [rasterio.crs module](https://rasterio.readthedocs.io/en/latest/api/rasterio.crs.html) only defines the `rasterio.crs.CRS` class. It is used in dataset objects objects and can also be used for various CRS manipulations.

The `rasterio.crs` submodule is imported automatically (how is this done?).

Note that `from_epsg()`, `from_dict()`, `from_string()`, and `from_wkt()` are called as static methods (techincally here they are class methods).


In [ ]:
dir(rasterio.crs)

### rasterio.transform module

The [rasterio.transform module](https://rasterio.readthedocs.io/en/latest/api/rasterio.transform.html) defines the `index()` and `xy()` methods of dataset classes via `TransformMethodsMixin` mixin class. It also defines a handfull of other functions but they are not obviously useful and may just be utility functions to support the mixin.

uses [Affine](https://github.com/sgillies/affine).

Note the `transform` property is type `affine.Affine` and has properties for accessing:

    a b c
    d e f
    g h i


### rasterio.path module

It's not clear what this is really about. It's probably just an internal utility module.


In [ ]:
parsed_path = rasterio.path.parse_path('https://opendap.cr.usgs.gov/opendap/hyrax/SRTMGL1.003/netcdf/N37E120.SRTMGL1.nc')

In [ ]:
parsed_path.is_remote
parsed_path.name
parsed_path.from_uri?

## Rasterio Topics

### Overviews

[Overviews](https://rasterio.readthedocs.io/en/latest/topics/overviews.html)

[Resampling](https://rasterio.readthedocs.io/en/latest/topics/resampling.html) may also be useful.

Overviews are discussed in [Working with Raster Datasets](https://geohackweek.github.io/raster/04-workingwithrasters/), and much of the same content is repeated in [Read Cloud Optimized Geotiffs](https://automating-gis-processes.github.io/CSC/notebooks/L5/read-cogs.html).

https://www.gdal.org/gdaladdo.html

https://gis.stackexchange.com/q/206509


### Tags

[Tagging datasets and bands](https://rasterio.readthedocs.io/en/latest/topics/tags.html)

Tags in rasterio include *metadata* in GDAL's data model. They are organized in *namespaces*, which reflect how GDAL metadata is split into *domains*.

`tags()`


---

## Scratch Mosaic

Describe how to mosaic files using `rasterio.merge.merge()`.

[Creating a raster mosaic | CSC](https://automating-gis-processes.github.io/CSC/notebooks/L5/raster-mosaic.html)

[Creating a raster mosaic | CSC18](https://automating-gis-processes.github.io/CSC18/lessons/L6/raster-mosaic.html)


In [ ]:
tif = rasterio.open('srtm/n37_w120_1arc_v3.tif')
tif.meta

In [ ]:
tif.bounds

In [ ]:
hgt = rasterio.open('srtm/N37W120.SRTMGL1.hgt.zip')
hgt.meta

In [ ]:
# hgt = rasterio.open('zip+https://e4ftl01.cr.usgs.gov/MEASURES/SRTMGL1.003/2000.02.11/N37W120.SRTMGL1.hgt.zip')

In [ ]:
from glob import glob

dem_fps = glob('srtm/*.tif')
src_files_to_mosaic = [rasterio.open(f) for f in dem_fps]

In [ ]:
src_files_to_mosaic

In [ ]:
from rasterio.merge import merge
mosaic, out_trans = merge(src_files_to_mosaic)

In [ ]:
from rasterio.plot import show
show(mosaic, cmap='terrain', transform=out_trans, vmin=0, interpolation='none')
mosaic.min(), mosaic.max()

In [ ]:
m = mosaic.copy()
# m[m==-32767].shape

m[m>0] = 0
m[m<0] = 32767

show(m, cmap='Reds', transform=out_trans, interpolation='none')
m.min(), m.max()

In [ ]:
m.shape

In [ ]:
hgt.width, hgt.height

In [ ]:
from rasterio.plot import show_hist
show_hist(
    m, bins=50, lw=0.0, stacked=False, alpha=0.3,
    histtype='stepfilled', title="Histogram")

## Scratch Landsat

In [ ]:
filepath = 'https://storage.googleapis.com/gcp-public-data-landsat/LC08/01/042/034/LC08_L1TP_042034_20190318_20190325_01_T1/LC08_L1TP_042034_20190318_20190325_01_T1_B4.TIF'
src = rasterio.open(filepath)

# This might be a 
with rasterio.open(filepath) as src:
    print(src.profile)

In [ ]:
src

In [ ]:
import rasterio.plot

In [ ]:
rasterio.plot.show(src)

In [ ]:
print(src.files)

In [ ]:
src.profile

In [ ]:
src.overviews(1)

In [ ]:
import matplotlib.pyplot as plt

fp = 'http://landsat-pds.s3.amazonaws.com/c1/L8/042/034/LC08_L1TP_042034_20170616_20170629_01_T1/LC08_L1TP_042034_20170616_20170629_01_T1_B4.TIF'
fp = 'http://landsat-pds.s3.amazonaws.com/c1/L8/042/034/LC08_L1TP_042034_20190318_20190325_01_T1/LC08_L1TP_042034_20190318_20190325_01_T1_B4.TIF'

# with rasterio.open(fp) as src:
#     # List of overviews from biggest to smallest
#     oviews = src.overviews(1)

#     # Retrieve the smallest thumbnail
#     oview = oviews[-1]
#     print('Decimation factor= {}'.format(oview))
#     # NOTE this is using a 'decimated read' (http://rasterio.readthedocs.io/en/latest/topics/resampling.html)
#     thumbnail = src.read(1, out_shape=(1, int(src.height // oview), int(src.width // oview)))

src =  rasterio.open(fp)
# List of overviews from biggest to smallest
oviews = src.overviews(1)

# Retrieve the smallest thumbnail
oview = oviews[-1]
print('Decimation factor= {}'.format(oview))
# NOTE this is using a 'decimated read' (http://rasterio.readthedocs.io/en/latest/topics/resampling.html)
thumbnail = src.read(1, out_shape=(1, int(src.height // oview), int(src.width // oview)))

print('array type: ',type(thumbnail))
print(thumbnail)

plt.imshow(thumbnail)
plt.colorbar()
plt.title('Overview - Band 4 {}'.format(thumbnail.shape))
plt.xlabel('Column #')
plt.ylabel('Row #')

In [ ]:
src.overviews(1)

In [ ]:
src.profile